In [ ]:
import sys
sys.path.append('../../')

import estraces
import scared
import matplotlib.pyplot as plt
from kresca.plot import CurvePlot
from kresca.synchronization import PatternSynchronization

## Import Traces

In [ ]:
ths = estraces.read_ths_from_ets_file('../../Traces/AES_CW_60k_unsync.ets')

In [ ]:
print(ths)

## Curve Plot

In [ ]:
#put inline=True if you want to display on notebook
CurvePlot(ths.samples[:5], inline=True) 

## Make a Pattern

In [ ]:
start_pattern = 350
end_pattern = 600
reference_trace = 0
pattern = ths[reference_trace].samples[start_pattern:end_pattern]

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(ths[reference_trace].samples, label='reference trace')
plt.plot(range(start_pattern, end_pattern), pattern, color='orange', label='pattern')
plt.legend(loc=2)
plt.show()

## Pattern Synchronization

In [ ]:
example = PatternSynchronization(
    ths=ths,
    patterns=[pattern],
    score_limit=0.5
)

In [ ]:
example.check()

In [ ]:
example.run("example.ets")